In [ ]:
# default_exp proxy

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# hide
!nbdev_build_lib --fname 11_Proxy_Request.ipynb

Converted 11_Proxy_Request.ipynb.


# TODO
- [ ] 抽象成class

# 代理爬虫
> 很多网站都有反爬虫机制，一个IP频繁访问一个网站，就会出现访问被拒绝的情况，所以换IP可以解决这个问题。（运用技术请克制，避免过度浪费服务器资源）

## 测试代理
> 有专用的付费代理IP稳定可靠，也有免费的代理IP可能会随时失效

In [ ]:
# export
import requests,json,re,random,sys,time,os
from bs4 import BeautifulSoup,Tag,NavigableString

from crawler_from_scratch.utils import *

from concurrent.futures import ThreadPoolExecutor
import pandas as pd


In [ ]:
# hide
url = 'https://www.baidu.com/'
headers={'user-agent':'Mozilla/5.0'}
proxies = {'https': '140.238.15.33:3128'}
res = requests.get(url,proxies=proxies,headers=headers,timeout=5)
res,res.text

(<Response [200]>,
 '<html>\r\n<head>\r\n\t<script>\r\n\t\tlocation.replace(location.href.replace("https://","http://"));\r\n\t</script>\r\n</head>\r\n<body>\r\n\t<noscript><meta http-equiv="refresh" content="0;url=http://www.baidu.com/"></noscript>\r\n</body>\r\n</html>')

## 制作代理池
> 网上有专门整理的代理池，可配置后直接调用：https://github.com/jhao104/proxy_pool
但原理是用他的代码，自己配置数据库，本地爬取数据，再封装函数自己去调用，大规模爬虫的时候可以用，现在就自己写个爬IP的爬虫吧

https://github.com/jiangxianli/ProxyIpLib

In [ ]:
class Proxy():
    '一个代理器，用爬取的免费代理ip，来爬取网站'
    def __init__(self):
        self.db = {}
        self.path = './data/11_Proxy.json'
        self.debug = True
        
        if os.path.exists(self.path):
            with open(self.path, 'r') as f:
                self.db = json.loads(f.read())
                print('加载成功',len(self.db.keys()))
        else:
            self.update(self)
    
    def update(self):
        '重新爬取ip，初始化health值，赋值给`self.db`，保存在`self.path`'
        data = []
        next_page_url = 'https://www.freeip.top/api/proxy_ips?page=1'
        while next_page_url:
            if self.debug: print('start:',next_page_url)
            res = requests.get(next_page_url)
            if res.status_code == 200:
                data_list = res.json()['data']['data']   
                data += data_list
                next_page_url = res.json()['data']['next_page_url']
            time.sleep(1)
        # ip list 转 dict 增加健康值
        self.db = {}
        for d in data:
            _id = d['unique_id']
            self.db[_id] = d
            self.db[_id]['health'] = 50
        with open(self.path, 'w') as f:
            json.dump(self.db,f)
            print('更新成功',len(self.db.keys()))
            
        self.validate('http://www.baidu.com/')
        self.validate('https://www.baidu.com/')
    
    def validate(self,url,max_workers=50):
        '批量测试ip有效性'
        protocol = url.split(':')[0]
        db_with_protocol = [self.db[k] for k in self.db if self.db[k]['protocol'] == protocol]
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            executor.map(lambda ip_obj : self._get(url,ip_obj),db_with_protocol)
        
    def choose_healthy_ip(self,protocol):
        '根据健康度，随机选择优质ip'
        db_with_protocol = [self.db[k] for k in self.db if self.db[k]['protocol'] == protocol]
        sorted_db = sorted(db_with_protocol, 
                           key = lambda item : item['health'],
                           reverse=True)
        return random.choice(sorted_db[:10])
        
    def update_ip_health(self,res,obj):
        '根据response，更新health'
        if res.status_code == 200:
            obj['health'] += 1
        else:
            obj['health'] = int(obj['health']/2)
        if self.debug: print(obj['ip'],'健康值变为：',obj['health'])
        
    def _get(self,url,ip_obj={}):
        '如果不指定ip，则自动选择`self.db`中最优的ip，访问网页，并更新health值'
        protocol = url.split(':')[0]
        if not ip_obj: ip_obj = self.choose_healthy_ip(protocol)
        ip = f"{ip_obj['protocol']}://{ip_obj['ip']}:{ip_obj['port']}"
        
        try:
            res = requests.get(url,
                               proxies={protocol: ip},
                               headers={'user-agent':'Mozilla/5.0'},
                               timeout=5)
        except:
            if self.debug: print(f'error: {ip}\n{sys.exc_info()}\n')
            res = requests.Response()
        
        self.update_ip_health(res,ip_obj)
            
        return res
    def get(self,url):
        '如果一个网页访问失败，会更换ip重试10次'
        try_times = 1
        while try_times < 11:
            if self.debug : print('\n',try_times,url)
            res = self._get(url)
            if res.status_code == 200:
                print('访问成功：',url)
                return res
            else:
                try_times += 1
        print('访问失败：',url)
        return res

## 自动切换代理

In [ ]:
# export
def proxy_get(url,ip,show_log=True):
    '根据ip代理访问'
    headers={'user-agent':'Mozilla/5.0'}
    
    protocol = 'https' if 'https' in ip else 'http'
    proxies = {protocol: ip}
    
    if show_log:
        print(ip,'代理中……')
    try:
        res = requests.get(url,proxies=proxies,headers=headers,timeout=5)
        return res
    except:
#         print(f'error: {sys.exc_info()}\n')
        return requests.Response()


In [ ]:
ip = random.choice(ip_pool)
res = proxy_get('https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91',ip)

https://186.183.220.2:8080 代理中……


## 校验&更替代理
> 成功的+1分，失败的减半，按照健康值获取前10名的ip

In [ ]:
# export
def init_ip_dict(ip_pool):
    '初始化所有ip的健康值'
    ip_dict = {}
    for ip in ip_pool:
        ip_dict[ip] = 50
    return ip_dict

In [ ]:
ip_dict = init_ip_dict(ip_pool)
len(ip_dict.keys())

150

In [ ]:
# export
def update_ip_health(res,ip,ip_dict):
    if res.status_code == 200:
        ip_dict[ip] += 1
#         print(ip,'健康值更新为：',ip_dict[ip])
        return True
    else:
        ip_dict[ip] = int(ip_dict[ip]/2)
#         print(ip,'健康值更新为：',ip_dict[ip])
        return False

def smart_proxy_get(url,ip_dict):
    '如果ip无效，可以自动更换，并记录ip健康值；如果更换10次都无效，则终止'
    try_times = 1
    while try_times < 11:
        # 选择健康值最好的10个ip
        ips = sorted(ip_dict,key=lambda k:ip_dict[k],reverse=True)[:10]
#         print(f'第{try_times}次尝试')
        ip = random.choice(ips)
        res = proxy_get(url,ip,show_log=False)
        if update_ip_health(res,ip,ip_dict):
            print('成功:',url)
            return res
        else:
#             print(res,res.text)
            try_times += 1
   
    print('失败:',url)
    return res

In [ ]:
# hide
smart_proxy_get('https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91',ip_dict)

第1次尝试
https://69.55.55.214:3128 代理中……
成功！


<Response [200]>

In [ ]:
# export
def validate_ip(ip,ip_dict,url='https://www.baidu.com/'):
    res = proxy_get(url,ip,show_log=False)
    update_ip_health(res,ip,ip_dict)

def validate_ip_dict(ip_dict,times=3,url='https://www.baidu.com/'):
    for i in range(times):
        with ThreadPoolExecutor(max_workers=50) as executor:
            executor.map(lambda ip : validate_ip(ip,ip_dict,url), ip_dict)
    return ip_dict

In [ ]:
ip_dict = validate_ip_dict(ip_dict,url='https://book.douban.com/')

https://128.199.242.16:44344 代理中……
https://186.183.220.2:8080 代理中……
https://180.183.30.94:8080 代理中……
https://117.88.5.78:3000 代理中……
https://3.125.122.212:8080 代理中……https://138.197.135.237:8080
 代理中……
https://180.183.136.27:8080 代理中……
https://209.141.46.133:8080 代理中……
https://190.61.57.210:999 代理中……
https://121.237.149.133:3000 代理中……
https://212.73.150.117:80 代理中……
https://134.249.156.228:82 代理中……
https://1.0.181.249:8080 https://139.99.46.133:6868代理中……
 https://181.196.151.82:49885代理中……
 https://143.202.78.148:999代理中…… 代理中……

https://218.27.136.169:8086 代理中……
https://125.27.252.210:8080 代理中……
https://183.88.106.147:8080https://128.199.172.205:8080  代理中……
https://116.196.87.86:20183代理中…… 代理中……

https://31.14.131.70:8080https://218.22.7.62:53281  代理中……
代理中……
https://181.129.165.102:999 代理中……
https://121.237.149.133:3000 健康值更新为： 5
https://14.115.107.33:808 代理中……
https://82.200.233.4:3128 代理中……
https://157.245.224.29:8080https://112.85.150.119:9999  https://103.228.117.244:8080代理中…… 代理中……


https://200.73.129.104:8080 健康值更新为： 5
https://78.153.120.33:39193 代理中……
https://200.73.129.104:3128 健康值更新为： 5
https://183.88.16.232:8080 代理中……
https://134.209.194.218:8080 健康值更新为： 5
https://180.210.201.57:3130 代理中……
https://128.199.227.116:47503 健康值更新为： 5
https://157.245.133.219:8080 代理中……
https://151.253.165.70:8080 健康值更新为： 5
https://200.73.129.104:80 代理中……
https://45.13.82.65:8080 健康值更新为： 5
https://118.96.125.137:8080 代理中……
https://212.34.130.195:3128 健康值更新为： 5
https://203.162.21.216:8000 代理中……
https://1.0.132.12:8080 健康值更新为： 5
https://121.237.148.216:3000 代理中……
https://128.199.242.16:44344 健康值更新为： 5
https://116.113.27.170:50035 健康值更新为： 5
https://203.113.103.54:8080https://167.179.4.134:8080 健康值更新为： 5
 健康值更新为： 5
https://70.165.64.210:48678 健康值更新为： 5
https://121.237.149.91:3000 健康值更新为： 5
https://58.8.141.128:8080 健康值更新为： 5
https://180.183.230.238:8080 健康值更新为： 5
https://206.189.84.152:44344 健康值更新为： 11
https://110.50.85.172:8089 健康值更新为： 5
https://119.57.108.89:53281 健康值更新为： 5
https://10

https://47.98.56.88:54321 健康值更新为： 2
https://167.179.4.134:8080 代理中……
https://103.228.117.244:8080 健康值更新为： 2
https://70.165.64.210:48678 代理中……
https://190.61.57.210:999 健康值更新为： 2
https://121.237.149.91:3000 代理中……
https://186.148.172.58:999 健康值更新为： 6
https://128.199.217.244:47503 代理中……
https://113.110.204.157:9000 健康值更新为： 2
https://103.134.220.192:3128 代理中……
https://111.118.128.161:8080 健康值更新为： 6
https://180.183.230.238:8080 代理中……
https://128.199.217.244:47503 健康值更新为： 2
https://206.189.84.152:44344 代理中……
https://200.73.129.61:3128 健康值更新为： 2
https://120.83.101.43:9999 代理中……
https://120.83.101.43:9999 健康值更新为： 2
https://142.93.130.169:8118 代理中……
https://128.199.242.16:44344 健康值更新为： 2
https://159.65.136.97:8080 代理中……
https://111.206.118.106:8080 健康值更新为： 2
https://110.50.85.172:8089 代理中……
https://180.183.0.49:8080 健康值更新为： 2
https://218.2.226.42:80 代理中……
https://159.65.136.97:8080 健康值更新为： 2
https://121.237.148.217:3000 代理中……
https://124.152.32.140:53281 健康值更新为： 2
https://119.57.108.89:53281 代理

https://111.118.128.161:8080 健康值更新为： 3
https://36.72.31.55:53281 代理中……
https://180.246.159.180:8080 健康值更新为： 1
https://175.139.179.65:39924 代理中……
https://212.73.150.117:80 健康值更新为： 1
https://109.198.110.234:8080 代理中……
https://128.199.242.16:44344 健康值更新为： 1
https://117.88.5.125:3000 代理中……
https://117.94.213.165:8118https://117.88.5.205:3000 健康值更新为： 1
https://110.5.102.130:3128 代理中……
 健康值更新为： 1
https://109.62.176.26:8080 代理中……
https://190.242.98.61:8083 健康值更新为： 1
https://179.49.24.102:3128 代理中……
https://163.125.68.155:8888 健康值更新为： 1
https://182.35.87.95:9999 代理中……
https://167.99.185.216:8080 健康值更新为： 1
https://180.183.99.2:8080 代理中……
https://36.72.31.55:53281 健康值更新为： 1
https://151.253.165.70:8080 代理中……
https://45.82.137.36:3128 健康值更新为： 1
https://222.128.9.235:33428 代理中……
https://103.28.121.58:3128 健康值更新为： 1
https://118.187.58.35:53281 代理中……
https://203.134.66.85:3128 健康值更新为： 1
https://58.8.141.128:8080 代理中……
https://109.198.110.234:8080 健康值更新为： 1
https://52.140.242.103:3128 代理中……
https://14

{'https://128.199.242.16:44344': 1,
 'https://186.183.220.2:8080': 1,
 'https://180.183.30.94:8080': 3,
 'https://117.88.5.78:3000': 1,
 'https://3.125.122.212:8080': 1,
 'https://138.197.135.237:8080': 1,
 'https://180.183.136.27:8080': 1,
 'https://209.141.46.133:8080': 1,
 'https://190.61.57.210:999': 1,
 'https://121.237.149.133:3000': 1,
 'https://212.73.150.117:80': 1,
 'https://134.249.156.228:82': 1,
 'https://1.0.181.249:8080': 1,
 'https://139.99.46.133:6868': 1,
 'https://181.196.151.82:49885': 1,
 'https://143.202.78.148:999': 3,
 'https://218.27.136.169:8086': 1,
 'https://125.27.252.210:8080': 1,
 'https://183.88.106.147:8080': 7,
 'https://128.199.172.205:8080': 1,
 'https://116.196.87.86:20183': 13,
 'https://31.14.131.70:8080': 2,
 'https://218.22.7.62:53281': 3,
 'https://181.129.165.102:999': 1,
 'https://14.115.107.33:808': 1,
 'https://82.200.233.4:3128': 1,
 'https://157.245.224.29:8080': 1,
 'https://112.85.150.119:9999': 1,
 'https://103.228.117.244:8080': 1,
 '

In [ ]:
# hide
for ip in ip_dict:
    if ip_dict[ip] > 1 :
        print(ip,ip_dict[ip])

https://180.183.30.94:8080 1
https://1.0.181.249:8080 1
https://116.196.87.86:20183 2
https://31.14.131.70:8080 2
https://218.22.7.62:53281 2
https://14.115.107.33:808 1
https://82.200.233.4:3128 1
https://112.85.150.119:9999 1
https://103.228.117.244:8080 1
https://103.28.121.58:3128 1
https://180.183.29.252:8080 1
https://134.236.160.66:8080 1
https://203.134.66.85:3128 1
https://186.148.172.58:999 1
https://125.123.143.240:9000 1
https://1.20.107.221:8080 1
https://180.183.0.49:8080 1
https://61.145.182.27:53281 1
https://165.227.220.35:3128 1
https://14.20.235.169:9797 1
https://183.88.6.202:8080 1
https://191.102.106.1:8181 1
https://58.251.230.21:9797 1
https://114.29.239.153:8080 1
https://159.203.44.177:3128 1
https://200.73.129.104:3128 1
https://180.253.69.233:8080 1
https://14.207.29.187:8080 1
https://111.206.118.106:8080 1
https://117.88.176.93:3000 1
https://45.82.137.36:3128 1
https://103.104.212.254:8080 3
https://111.118.128.161:8080 1
https://117.88.5.205:3000 1
https

In [ ]:
# export
def get_healthy_ip_pool(protocol='https',update_now=False):
    if update_now:
        update_ip_pool(protocol)
    ip_list = load_ip_pool()
    ip_dict = init_ip_dict(ip_list)
    validate_ip_dict(ip_dict)
    return ip_dict

In [ ]:
get_healthy_ip_pool('https')

IP数: 144
http://84.17.47.192:80 代理中……
http://103.233.122.132:8080 代理中……
http://80.187.140.26:80 代理中……
http://118.69.50.154:80 代理中……
http://84.17.47.190:80 代理中……
http://112.80.248.95:80 代理中……
http://117.185.16.253:80http://119.41.236.180:8010  代理中……
http://61.135.185.112:80代理中…… 
http://52.80.58.248:3128代理中…… 代理中……
http://65.52.158.214:80
 代理中……
http://111.206.37.68:80 代理中……http://61.135.186.243:80 
代理中……http://163.172.152.52:8811
 代理中……http://84.17.47.204:80
 代理中……
http://67.205.132.241:8080 http://103.44.29.22:80代理中……
 代理中……http://180.97.33.218:80
 代理中……
http://159.203.166.41:8080 代理中……
http://185.186.77.197:80 代理中……
http://84.17.47.185:80 代理中……
http://111.206.37.100:80 代理中……
http://39.137.95.71:80http://61.135.185.153:80  代理中……
代理中……
http://61.135.185.152:80 http://111.206.37.244:80 代理中……
代理中……
http://51.158.165.18:8811 代理中……
http://39.137.95.70:80 http://188.226.141.127:8080 代理中……
代理中……
http://58.220.95.86:9401 代理中……
http://148.251.153.6:1080 http://61.135.185.69:80代理中……
 http://198

http://84.17.47.182:80 健康值更新为： 51
http://138.197.222.35:8080 健康值更新为： 51
http://84.17.47.197:80 健康值更新为： 51
http://37.120.159.66:80 健康值更新为： 25
http://84.17.47.192:80 代理中……
http://103.233.122.132:8080 代理中……
http://80.187.140.26:80http://118.69.50.154:80http://84.17.47.190:80  代理中……
http://112.80.248.95:80代理中……  
http://117.185.16.253:80代理中……
 代理中……
http://119.41.236.180:8010代理中…… 代理中……
http://61.135.185.112:80
 代理中……
http://52.80.58.248:3128 代理中……
http://65.52.158.214:80 代理中……
http://111.206.37.68:80 http://61.135.186.243:80代理中…… 代理中……

http://163.172.152.52:8811 http://84.17.47.204:80代理中……
 代理中……
http://67.205.132.241:8080http://103.44.29.22:80 http://180.97.33.218:80 代理中……
 代理中……
代理中……
http://159.203.166.41:8080http://185.186.77.197:80  代理中……
代理中……
http://84.17.47.185:80 http://111.206.37.100:80 http://39.137.95.71:80代理中……
 代理中……
代理中……
http://61.135.185.153:80http://61.135.185.152:80  代理中……
代理中……http://111.206.37.244:80http://51.158.165.18:8811  代理中……
代理中……
http://39.137.95.70:80 http:/

http://123.125.115.74:80 健康值更新为： http://163.177.151.224:8026 http://180.97.33.144:80
健康值更新为： http://163.172.189.32:881152
 http://183.56.161.62:80 健康值更新为： 52
http://173.249.44.18:80 健康值更新为： 健康值更新为： 26
 52健康值更新为：
 26
http://178.128.126.135:44344 健康值更新为： 26
http://111.206.37.161:80http://51.79.85.125:8080  健康值更新为： 26
健康值更新为： 26
http://51.158.186.242:8811http://51.158.172.165:8811http://162.243.108.161:8080 健康值更新为：http://39.137.95.71:8080   http://84.17.47.202:80 健康值更新为：健康值更新为：http://138.197.222.35:8080http://51.158.108.135:8811 健康值更新为： 26
   http://185.186.77.115:80健康值更新为：26健康值更新为： 健康值更新为： 26
 http://61.135.185.68:8026
 
http://39.137.95.69:80 健康值更新为： 26
26
 26
52健康值更新为： 
http://181.30.28.178:80健康值更新为：  52 26
健康值更新为：
 26
http://123.125.115.242:80 健康值更新为： http://58.240.97.154:3128http://52.140.242.103:3128 26 
健康值更新为： 26
健康值更新为： 26
http://218.60.8.99:3129http://180.97.104.97:80http://84.17.47.197:80http://37.120.159.63:80 健康值更新为： 26
  http://165.227.84.169:80健康值更新为：  健康值更新为： 26
 健康值更新为：ht

{'http://84.17.47.192:80': 53,
 'http://103.233.122.132:8080': 53,
 'http://80.187.140.26:80': 53,
 'http://118.69.50.154:80': 53,
 'http://84.17.47.190:80': 53,
 'http://112.80.248.95:80': 53,
 'http://117.185.16.253:80': 53,
 'http://119.41.236.180:8010': 53,
 'http://61.135.185.112:80': 53,
 'http://52.80.58.248:3128': 26,
 'http://65.52.158.214:80': 53,
 'http://111.206.37.68:80': 53,
 'http://61.135.186.243:80': 53,
 'http://163.172.152.52:8811': 53,
 'http://84.17.47.204:80': 53,
 'http://67.205.132.241:8080': 53,
 'http://103.44.29.22:80': 53,
 'http://180.97.33.218:80': 26,
 'http://159.203.166.41:8080': 26,
 'http://185.186.77.197:80': 53,
 'http://84.17.47.185:80': 26,
 'http://111.206.37.100:80': 53,
 'http://39.137.95.71:80': 53,
 'http://61.135.185.153:80': 26,
 'http://61.135.185.152:80': 26,
 'http://111.206.37.244:80': 26,
 'http://51.158.165.18:8811': 26,
 'http://39.137.95.70:80': 26,
 'http://188.226.141.127:8080': 26,
 'http://58.220.95.86:9401': 26,
 'http://148.25

## 用代理爬取豆瓣页面
> 昨天用豆瓣页面测试爬虫功能的时候，就发现了访问频率过高的被拒的问题，今天就用爬虫来抓取整个互联网类目下的图书信息

In [ ]:
#hide
def get_douban_data(i,data,ip_dict):
    url = f'https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start={i}&type=T'
    print(url)
    res = smart_proxy_get(url,ip_dict)    

    if res.status_code == 200:
        soup = BeautifulSoup(res.text)
        main_content = soup.body.find('ul',class_='subject-list')

        for c in get_children(main_content):
            item_data = get_data(c)
            a_nbg_url = item_data['a_nbg_url']
            _id = re.search(r'/(\d+)/',a_nbg_url).group(1)
            # 写入data
            data[_id] = item_data
        
    else:
        print(res,res.text)

In [ ]:
#hide
data = {}
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(lambda i : get_douban_data(i,data,ip_dict), range(0,1000,20)) 


https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=0&type=Thttps://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=20&type=T

第1次尝试
https://31.14.131.70:8080 代理中……
第1次尝试https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=40&type=T

第1次尝试
https://180.183.29.252:8080 代理中……
https://165.227.220.35:3128 代理中……
https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=60&type=Thttps://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=80&type=T
第1次尝试
https://1.20.107.221:8080 代理中……

第1次尝试https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=100&type=Thttps://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=120&type=T
第1次尝试
https://31.14.131.70:8080 代理中……

第1次尝试
https://31.14.131.70:8080 代理中……

https://180.183.30.94:8080 代理中……
https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=140&type=T
https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=160&type=T第1次尝试

https://61.145.182.27:53281 代理中……
第1次尝试
https://31.14.131.70:8080 代理中……


https://163.125.68.155:8888 健康值更新为： 1
成功！
https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=760&type=T
第1次尝试
https://157.245.251.213:80 代理中……
https://190.242.98.61:8083 健康值更新为： 0
第3次尝试
https://31.14.131.70:8080 代理中……
https://200.73.129.104:3128 健康值更新为： 0
第5次尝试
https://212.73.150.117:8080 代理中……
https://103.104.212.254:8080 健康值更新为： 3
成功！
https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=780&type=T
第1次尝试
https://111.118.128.161:8080 代理中……
https://157.245.251.213:80 健康值更新为： 9
成功！
https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=800&type=T
第1次尝试
https://185.229.236.145:8080 代理中……
https://185.229.236.145:8080 健康值更新为： 0
第4次尝试
https://103.104.212.254:8080 代理中……
https://190.242.98.61:8083 健康值更新为： 0
第4次尝试
https://212.73.150.117:8080 代理中……
https://185.229.236.145:8080 健康值更新为： 0
第4次尝试
https://31.14.131.70:8080 代理中……
https://31.14.131.70:8080 健康值更新为： 7
成功！
https://book.douban.com/tag/%E4%BA%92%E8%81%94%E7%BD%91?start=820&type=T
第1次尝试
https://31.14.131.70:8080 代理

In [ ]:
# hide
dataframe = pd.DataFrame.from_dict(data,orient='index')
dataframe.head()

,a_nbg_url,img_no_class_src,a_no_class_url,a_no_class_title,a_no_class_text,span_no_class_text,div_pub_text,span_rating_nums_text,span_pl_text,p_no_class_text
34908331,https://book.douban.com/subject/34908331/,https://img9.doubanio.com/view/subject/s/publi...,https://book.douban.com/subject/34908331/?chan...,交往在云端,在豆瓣购买,: 数字时代的人际关系,[美] 南希·K.拜厄姆 / 董晨宇、唐悦哲 / 中国人民大学出版社 / 2020,8.7,(60人评价),互联网和移动电话颠覆了我们对于自身和人际关系的许多传统认知。说起它们对生活的影响，我们既焦虑...
25942507,https://book.douban.com/subject/25942507/,https://img9.doubanio.com/view/subject/s/publi...,https://read.douban.com/ebook/5740451/?dcs=tag...,参与感,去看电子版,: 小米口碑营销内部手册,黎万强 / 中信出版社 / 2014-8 / 56.00元,7.5,(9176人评价),◆雷军亲笔作序，小米联合创始人黎万强著。\n◆揭开小米4年600亿奇迹背后的理念、方法和案例...
33474750,https://book.douban.com/subject/33474750/,https://img9.doubanio.com/view/subject/s/publi...,https://book.douban.com/subject/33474750/buylinks,浪潮之巅（第四版）（上下册）,纸质版 69.50 元起,NaN,吴军 / 人民邮电出版社 / 2019-6 / 139.00元,9.1,(304人评价),这不只是一部科技产业发展历史集……\n更是在这个智能时代，一部IT人非读不可，而非IT人也应...
20429677,https://book.douban.com/subject/20429677/,https://img9.doubanio.com/view/subject/s/publi...,https://read.douban.com/ebook/29224686/?dcs=ta...,大数据时代,去看电子版,: 生活、工作与思维的大变革,[英] 维克托•迈尔•舍恩伯格（Viktor Mayer-Schönberger） / 周涛...,7.5,(16135人评价),《大数据时代》是国外大数据研究的先河之作，本书作者维克托•迈尔•舍恩伯格被誉为“大数据商业应...
10945606,https://book.douban.com/subject/10945606/,https://img9.doubanio.com/view/subject/s/publi...,https://read.douban.com/ebook/484790/?dcs=tag-...,精益创业,去看电子版,: 新创企业的成长思维,[美] 埃里克·莱斯 / 吴彤 / 中信出版社 / 2012-8 / 49.00元,8.3,(4759人评价),《精益创业:新创企业的成长思维》内容简介：我们正处在一个空前的全球创业兴盛时代，但无数创业公...


得到了这样的规范的数据结构的数据，就可以进行很多数据分析的工作，比如评价数分布，评分分布，热门作者等等

In [ ]:
# hide
dataframe.describe()

,a_nbg_url,img_no_class_src,a_no_class_url,a_no_class_title,a_no_class_text,span_no_class_text,div_pub_text,span_rating_nums_text,span_pl_text,p_no_class_text
count,960,960,960,959,960,554,957,887,960,909
unique,960,959,960,918,428,543,957,52,571,902
top,https://book.douban.com/subject/10785583/,https://img9.doubanio.com/f/shire/5522dd1f5b74...,https://read.douban.com/ebook/54346394/?dcs=ta...,失控,去看电子版,: 全人类的最终命运和结局,[美] 艾伯特-拉斯洛·巴拉巴西 / 马慧 / 中国人民大学出版社 / 2012-6 / 5...,8.0,(少于10人评价),一个企业的发展与崛起，绝非只是空有领导强人即可达成。任何的决策、同期的商业环境，都在都影响着...
freq,1,2,1,3,307,3,1,49,62,2
